In [1]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
import matplotlib.pyplot as plt
from torch.utils.data import TensorDataset
import numpy as np

In [2]:
import pickle
modelName = '../models/features2021-12-14.model'
with open(modelName,'rb') as f:
        feature_extractor = pickle.load(f)

/opt/anaconda3/lib/python3.9/site-packages/sklearn/base.py:310: UserWarning: Trying to unpickle estimator CountVectorizer from version 1.0.1 when using version 0.24.2. This might lead to breaking code or invalid results. Use at your own risk.
  warnings.warn(
/opt/anaconda3/lib/python3.9/site-packages/sklearn/base.py:310: UserWarning: Trying to unpickle estimator TfidfTransformer from version 1.0.1 when using version 0.24.2. This might lead to breaking code or invalid results. Use at your own risk.
  warnings.warn(


In [3]:
import pandas as pd
train_csv_path = '../data/mbti_full_pull_half_train.csv'
# train_csv_path = '/mnt/c/Users/haiya/Downloads/finalp/mbti_full_pull_half_train.csv'

train_df = pd.read_csv(train_csv_path, index_col=0)


In [11]:
test_df = pd.read_csv('../data/mbti_full_pull_half_test.csv').sample(5000)

In [8]:
new_indices = []
for k,group in train_df.groupby(["mbti_type"]).groups.items():
    if len(group) > 10000:
        new_indices.extend(group[:10000])
    else:
        new_indices.extend(group)
train_df = train_df.loc[new_indices]
train_df.groupby(['mbti_type']).count()

,body
mbti_type,
ENFJ,3279
ENFP,10000
ENTJ,6614
ENTP,10000
ESFJ,346
ESFP,695
ESTJ,1043
ESTP,3757
INFJ,10000


In [10]:
X_train_feat = feature_extractor.get_features(train_df.body)


NameError: name 'test_df' is not defined

In [12]:
X_test_feat = feature_extractor.get_features(test_df.body)


In [14]:
mbtis = sorted(train_df.mbti_type.unique())
mbti2int = {v:i for i,v in enumerate(mbtis)}
int2mbti = {i:v for i,v in enumerate(mbtis)}

y_train = train_df.mbti_type.apply(lambda x: mbti2int[x])
y_test = test_df.mbti_type.apply(lambda x: mbti2int[x])


In [ ]:
import os

In [21]:
first_layers = dict()
model_path = '../models/first_layer/'
for model in os.listdir(model_path):
    print(model_path+model)    
    with open(model_path+model,'rb') as f:
        first_layers[model[:4]] = pickle.load(f)
    

../models/first_layer/
../models/first_layer/
../models/first_layer/
../models/first_layer/
../models/first_layer/
../models/first_layer/
../models/first_layer/
../models/first_layer/
../models/first_layer/
../models/first_layer/
../models/first_layer/
../models/first_layer/
../models/first_layer/
../models/first_layer/


/opt/anaconda3/lib/python3.9/site-packages/sklearn/base.py:310: UserWarning: Trying to unpickle estimator DecisionTreeClassifier from version 1.0.1 when using version 0.24.2. This might lead to breaking code or invalid results. Use at your own risk.
  warnings.warn(
/opt/anaconda3/lib/python3.9/site-packages/sklearn/base.py:310: UserWarning: Trying to unpickle estimator RandomForestClassifier from version 1.0.1 when using version 0.24.2. This might lead to breaking code or invalid results. Use at your own risk.
  warnings.warn(


UnpicklingError: invalid load key, '\x00'.

In [ ]:
 def first_layer_output(self,series):
    return np.array([
        self.first_layer[name].predict_proba(series)[:,0] 
        for name in list(sorted(self.first_layer.keys()))]).T

In [ ]:
help(DataLoader)

Help on class DataLoader in module torch.utils.data.dataloader:

class DataLoader(typing.Generic)
 |  DataLoader(dataset: torch.utils.data.dataset.Dataset[+T_co], batch_size: Optional[int] = 1, shuffle: bool = False, sampler: Optional[torch.utils.data.sampler.Sampler] = None, batch_sampler: Optional[torch.utils.data.sampler.Sampler[Sequence]] = None, num_workers: int = 0, collate_fn: Optional[Callable[[List[~T]], Any]] = None, pin_memory: bool = False, drop_last: bool = False, timeout: float = 0, worker_init_fn: Optional[Callable[[int], NoneType]] = None, multiprocessing_context=None, generator=None, *, prefetch_factor: int = 2, persistent_workers: bool = False)
 |  
 |  Data loader. Combines a dataset and a sampler, and provides an iterable over
 |  the given dataset.
 |  
 |  The :class:`~torch.utils.data.DataLoader` supports both map-style and
 |  iterable-style datasets with single- or multi-process loading, customizing
 |  loading order and optional automatic batching (collation) 

In [ ]:
X = torch.as_tensor(X,dtype=torch.float)
y = torch.as_tensor(y)

train_dl = DataLoader(TensorDataset(X,y),64)

TypeError: as_tensor(): argument 'dtype' must be torch.dtype, not str

[ ] 同理, 创建一个test_dl

接下来做的模型训练直接使用cpu算

In [ ]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(80, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 16)
        )

    def forward(self, x):
        logits = self.linear_relu_stack(x)
        return logits

model = NeuralNetwork()
print(model)

In [ ]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)

In [ ]:
def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)

        # Compute prediction error
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % 100 == 0:
            loss, current = loss.item(), batch * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

epochs = 5
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(train_dl, model, loss_fn, optimizer)
    test(test_dl, model, loss_fn)
print("Done!")



In [ ]:
torch.save(model.state_dict(), "../models/2model.pth")